In [7]:
import pandas as pd
import dask.dataframe as dd
from dask.multiprocessing import get
import numpy as np
import timeit

In [26]:
data = pd.DataFrame()
data['col1'] = np.random.normal(size = 1000000)
data['col2'] = np.random.normal(size = 1000000)



#========
#========
ddata = dd.from_pandas(data, npartitions=90)
def myfunc(x,y): return y*(x**2+1)
def apply_myfunc_to_DF(df): return df.apply((lambda row: myfunc(*row)), axis=1)
def pandas_apply(): return apply_myfunc_to_DF(data)
def dask_apply(): return ddata.map_partitions(apply_myfunc_to_DF).compute(scheduler='multiprocessing')  


#t_pds = timeit.Timer(lambda: pandas_apply())
#print(t_pds.timeit(number=1))

t_dsk = timeit.Timer(lambda: dask_apply())
print(t_dsk.timeit(number=1))

4.148672234965488


In [28]:
'''
OPEN DATAFRAME OF dbNSFP
'''
def get_dbSNP_version(dataframe):
    for column in list(dataframe):
        if "rs_dbSNP" in column:
            return column[-3:]
def get_hg_version(dataframe):
    bigger_value = "0"
    for column in list(dataframe):
        if(("hg" in column) and (column[2:4] > bigger_value)):
            bigger_value = column[2:4]
    return bigger_value
        
FILENAME = "dbNSFP3.5a_variant.chrM"
COLUMN_NAMES = ["genename", "rs_dbSNP150", "hg19_pos(1-based)", "cds_strand", "codonpos", "codon_degeneracy",
                "ref", "alt", "aaref", "aaalt", "aapos", "Ensembl_geneid", "Ensembl_transcriptid",
                "Ensembl_proteinid", "Uniprot_acc_Polyphen2", "Transcript_id_VEST3", "SIFT_pred",
                "Polyphen2_HDIV_pred", "Polyphen2_HVAR_pred", "LRT_pred", "MutationTaster_pred",
                "MutationAssessor_pred", "FATHMM_pred", "PROVEAN_pred", "VEST3_score", "MetaSVM_pred",
                "MetaLR_pred", "M-CAP_pred", "REVEL_score", "MutPred_score", "CADD_phred", "DANN_score",
                "fathmm-MKL_coding_pred"]

pd.set_option('display.max_columns', 300)

'''
TO RUN ON LIKA COMPUTER, USE THE SECOND READ_CSV FUNCTION
'''
#df = pd.read_csv(FILENAME, sep='\t')
df = pd.read_csv("/bioinfo/dbNSFPv3.5a/" + FILENAME, sep='\t')

#df = df[COLUMN_NAMES]

#df = df.loc[df['rs_dbSNP150'] != '.']
get_hg_version(df)

'19'

In [29]:
'''
OUTPUT IS A DICT OF PERCENT OF INFORMATION FOR EACH COLUMN
'''
def count_blank_information():
    for column_index in range(len(row.index)):
        value = df.iloc[row_index].values[column_index]
        if type(value) == str:
            if((value == '.') or ("./" in value)):             
                info_dict[row.index[column_index]] += 1

ddata = dd.from_pandas(df, npartitions=90)
info_dict = {}


for column_name in list(df):
    info_dict[column_name] = 0

                
for i in info_dict:
    info_dict[i] = str(round((1 - info_dict[i]/df.shape[0]) * 100),2) + '%'
info_dict

KeyboardInterrupt: 